### Keras Sequential/Functional API 模式建立模型 

最典型和常用的神经网络结构是将一堆层按特定顺序叠加起来，那么，我们是不是只需要提供一个层的列表，就能由 Keras 将它们自动首尾相连，形成模型呢？Keras 的 Sequential API 正是如此。通过向 `tf.keras.models.Sequential()` 提供一个层的列表，就能快速地建立一个 `tf.keras.Model` 模型并返回：

In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(100, activation=tf.nn.relu),
            tf.keras.layers.Dense(10),
            tf.keras.layers.Softmax()
        ])

不过，这种层叠结构并不能表示任意的神经网络结构。为此，Keras 提供了 Functional API，帮助我们建立更为复杂的模型，例如多输入 / 输出或存在参数共享的模型。其使用方法是将层作为可调用的对象并返回张量（这点与之前章节的使用方法一致），并将输入向量和输出向量提供给 `tf.keras.Model` 的 `inputs` 和 `outputs` 参数，示例如下：

In [3]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(units=10)(x)
outputs = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

### 使用 Keras Model 的 `compile` 、 `fit` 和 `evaluate` 方法训练和评估模型

In [ ]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(units=10)(x)
outputs = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [4]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

`tf.keras.Model.compile` 接受 3 个重要的参数：

> - `oplimizer` ：优化器，可从 `tf.keras.optimizers` 中选择；
> - `loss` ：损失函数，可从 `tf.keras.losses` 中选择；
> - `metrics` ：评估指标，可从 `tf.keras.metrics` 中选择。

In [5]:
model.fit(data_loader.train_data, data_loader.train_label,epochs=num_epochs, batch_size=batch_size)

NameError: name 'data_loader' is not defined

`tf.keras.Model.fit` 接受 5 个重要的参数：

> - `x` ：训练数据；
> - `y` ：目标数据（数据标签）；
> - `epochs` ：将训练数据迭代多少遍；
> - `batch_size` ：批次的大小；
> - `validation_data` ：验证数据，可用于在训练过程中监控模型的性能。

Keras 支持使用 `tf.data.Dataset` 进行训练，详见 [tf.data](https://tf.wiki/en/basic/tools.html#tfdata) 。

最后，使用 `tf.keras.Model.evaluate` 评估训练效果，提供测试数据及标签即可：

In [ ]:
 print(model.evaluate(data_loader.test_data, data_loader.test_label))

## 自定义层、损失函数和评估指标 *

可能你还会问，如果现有的这些层无法满足我的要求，我需要定义自己的层怎么办？事实上，我们不仅可以继承 `tf.keras.Model` 编写自己的模型类，也可以继承 `tf.keras.layers.Layer` 编写自己的层。

### 自定义层 

自定义层需要继承 `tf.keras.layers.Layer` 类，并重写 `__init__` 、 `build` 和 `call` 三个方法，如下所示：

In [ ]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        # 初始化代码

    def build(self, input_shape):     # input_shape 是一个 TensorShape 类型对象，提供输入的形状
        # 在第一次使用该层的时候调用该部分代码，在这里创建变量可以使得变量的形状自适应输入的形状
        # 而不需要使用者额外指定变量形状。
        # 如果已经可以完全确定变量的形状，也可以在__init__部分创建变量
        self.variable_0 = self.add_weight(...)
        self.variable_1 = self.add_weight(...)

    def call(self, inputs):
        # 模型调用的代码（处理输入并返回输出）
        return output

例如，如果我们要自己实现一个 [本章第一节](https://tf.wiki/zh/basic/models.html#linear) 中的全连接层（ `tf.keras.layers.Dense` ），可以按如下方式编写。此代码在 `build` 方法中创建两个变量，并在 `call` 方法中使用创建的变量进行运算：

In [6]:
class LinearLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):     # 这里 input_shape 是第一次运行call()时参数inputs的形状
        self.w = self.add_variable(name='w',
            shape=[input_shape[-1], self.units], initializer=tf.zeros_initializer())
        self.b = self.add_variable(name='b',
            shape=[self.units], initializer=tf.zeros_initializer())

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [7]:
class LinearModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer = LinearLayer(units=1)

    def call(self, inputs):
        output = self.layer(inputs)
        return output

### 自定义损失函数和评估指标 

自定义损失函数需要继承 `tf.keras.losses.Loss` 类，重写 `call` 方法即可，输入真实值 `y_true` 和模型预测值 `y_pred` ，输出模型预测值和真实值之间通过自定义的损失函数计算出的损失值。下面的示例为均方差损失函数：

In [8]:
class MeanSquaredError(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_pred - y_true))

自定义评估指标需要继承 `tf.keras.metrics.Metric` 类，并重写 `__init__` 、 `update_state` 和 `result` 三个方法。下面的示例对前面用到的 `SparseCategoricalAccuracy` 评估指标类做了一个简单的重实现：

In [9]:
class SparseCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self):
        super().__init__()
        self.total = self.add_weight(name='total', dtype=tf.int32, initializer=tf.zeros_initializer())
        self.count = self.add_weight(name='count', dtype=tf.int32, initializer=tf.zeros_initializer())

    def update_state(self, y_true, y_pred, sample_weight=None):
        values = tf.cast(tf.equal(y_true, tf.argmax(y_pred, axis=-1, output_type=tf.int32)), tf.int32)
        self.total.assign_add(tf.shape(y_true)[0])
        self.count.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.count / self.total